In [123]:
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import numpy as np
import random
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("michellejieli/emotion_text_classifier")
model = AutoModelForSequenceClassification.from_pretrained("michellejieli/emotion_text_classifier")


In [134]:
FER2013 = load_model('FER2013_BestYet.keras')
AffectNET = load_model('AffectNet_Final.keras')
SER = load_model('best_weights.keras')
emotionMapping = {'0': 'anger','1': 'fear','2': 'happy', '3': 'sad', '4': 'surprise', '5': 'neutral'}


In [135]:
print('SER input shape: ',SER.input_shape)
print('FER2013 input shape: ',FER2013.input_shape)
print('AffectNET input shape: ',AffectNET.input_shape)


SER input shape:  (None, 335, 50)
FER2013 input shape:  (None, 48, 48, 1)
AffectNET input shape:  (None, 96, 96, 3)


In [136]:
def fer2013inputshape(img):
    img = cv2.resize(img, (48, 48))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)
    return img

In [137]:
def AffectNetinputshape(img):
    img = cv2.resize(img, (96, 96))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)
    return img

In [256]:
#SER input shape:  (None, 335, 50)
def SERinputshape(wav):
    #wav = wav[0:335]
    wav = np.pad(wav, (0, 335-len(wav)), 'constant', constant_values=(0, 0))
    wav = np.expand_dims(wav, axis=-1)
    wav = np.expand_dims(wav, axis=0)
    return wav

    

In [278]:
img = cv2.imread("/home/group02-f24/Desktop/SadBaby.jpg")
img = img.astype('float32') / 255.0  # Normalize if needed

In [279]:
FER2013img = fer2013inputshape(img)
AffectNETimg = AffectNetinputshape(img)

### RAW PREDICTIONS

In [ ]:
FER2013pred = FER2013.predict(FER2013img)
AffectNETpred = AffectNET.predict(AffectNETimg)
AffectNETpred = 1.1*AffectNETpred
FER2013pred = 0.9*FER2013pred
concPred = FER2013pred + AffectNETpred
print('FER2013 prediction: ',FER2013pred)
print('AffectNET prediction: ',AffectNETpred)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
FER2013 prediction:  [[0.49917305 0.10686404 0.20733239 0.02033958 0.01804267 0.14824827]]
AffectNET prediction:  [[0.03042624 0.15741353 0.01259953 0.57219344 0.11132995 0.11603738]]


### Normalized PREDICTIONS

In [284]:
FER2013pred = np.argmax(FER2013pred, axis=1)
AffectNETpred = np.argmax(AffectNETpred, axis=1)
print('FER2013 prediction: ',emotionMapping[str(FER2013pred[0])])
print('AffectNET prediction: ',emotionMapping[str(AffectNETpred[0])])

FER2013 prediction:  anger
AffectNET prediction:  sad


### Concatenated Predictions

In [285]:
finalPrediction = np.argmax(concPred, axis=1)
print('FER2013 prediction: ',emotionMapping[str(finalPrediction[0])])


FER2013 prediction:  sad
